In [ ]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

from firedrake.eigensolver import LinearEigenproblem, LinearEigensolver

# Stokes Eigenvalue Problem with BDM-DG Elements

Solving the Stokes eigenvalue problem on a unit square domain with 0 Dirichlet boundary conditions.
Using degree-1 BDM elements for velocity and degree-0 DG elements for pressure, with a symmetric interior penalty method.

In [ ]:
# 1. Mesh Definition
# Unit square mesh with N x N elements
N = 16
mesh = UnitSquareMesh(N, N)

In [ ]:
# 2. Function Spaces
# Velocity: BDM (Brezzi-Douglas-Marini) degree 1
# Pressure: DG (Discontinuous Galerkin) degree 0
# This pair is inf-sup stable.
V = FunctionSpace(mesh, "BDM", 1)
Q = FunctionSpace(mesh, "DG", 0)
W = V * Q

# 3. Trial and Test Functions
u, p = TrialFunctions(W)
v, q = TestFunctions(W)

# 4. Solution Function
w = Function(W)
u_sol, p_sol = w.subfunctions

In [ ]:
# 5. Problem Parameters
nu = Constant(0.01)  # Kinematic viscosity
# No forcing term for eigenvalue problem

In [ ]:
# 6. Variational Form
# We use a Symmetric Interior Penalty (SIP) method for the viscous term
# because BDM elements are H(div)-conforming but not H1-conforming.

n = FacetNormal(mesh)
h = CellDiameter(mesh)
sigma = Constant(10.0)  # Penalty parameter for SIPG

def a_viscous(u, v):
    # Volume term: nu * grad(u) : grad(v)
    term = nu * inner(grad(u), grad(v)) * dx
    
    # Interior Facets
    term -= nu * inner(avg(grad(u)), jump(v, n)) * dS
    term -= nu * inner(avg(grad(v)), jump(u, n)) * dS
    term += (sigma / avg(h)) * nu * inner(jump(u, n), jump(v, n)) * dS
    
    # Boundary Facets (Weak imposition of no-slip BC u=0)
    term -= nu * inner(grad(u), outer(v, n)) * ds
    term -= nu * inner(grad(v), outer(u, n)) * ds
    term += (sigma / h) * nu * inner(u, v) * ds
    
    return term

def b_pressure(u, p, v, q):
    # - (p, div(v)) + (q, div(u))
    return -p * div(v) * dx + q * div(u) * dx

# Stiffness matrix form (LHS)
# A(u, p; v, q)
A = a_viscous(u, v) + b_pressure(u, p, v, q)

# Mass matrix form (RHS)
# M(u, p; v, q) = (u, v)
# We look for eigenvalues lambda such that A x = lambda M x
M = inner(u, v) * dx

In [ ]:
# 7. Solver
# We solve for the eigenvalues of the Stokes operator.
# We look for the smallest magnitude eigenvalues.

# Note: Since we use weak BCs, we pass bcs=None.
eigenproblem = LinearEigenproblem(A, M, bcs=None)

# Request 6 eigenvalues
n_evals = 6
solver = LinearEigensolver(eigenproblem, n_evals, solver_parameters={
    "eps_type": "krylovschur",
    "eps_target": 0.0, # Look near 0
    "eps_which": "TARGET_MAGNITUDE", # Smallest magnitude
    "st_type": "sinvert", # Spectral transformation
    "st_pc_type": "lu",
    "st_pc_factor_mat_solver_type": "mumps"
})

n_found = solver.solve()
print(f"Found {n_found} eigenvalues.")

In [ ]:
# 8. Output
for i in range(n_found):
    lam = solver.eigenvalue(i)
    print(f"Eigenvalue {i}: {lam}")

# Save the first eigenfunction
# eigenfunction(i) returns (real_part, imag_part)
u_mode, p_mode = solver.eigenfunction(0)[0].subfunctions 
u_mode.rename("VelocityMode0")
p_mode.rename("PressureMode0")

outfile = File("stokes_eigenvectors.pvd")
outfile.write(u_mode, p_mode)
print("Saved first eigenmode to stokes_eigenvectors.pvd")